In [91]:
import pandas as pd
import numpy as np
from item.item_list import (
    ItemList,
    Item
)
from item.clustering.utils import (
    load_clustering_results_pickle,
    load_clustering_results
)
from collections import defaultdict
import re

In [51]:
results, outliers = load_clustering_results_pickle('../data/output/druid_fasttext/')

itemlist = ItemList()
itemlist.load_items_from_file('../data/output/druid_fasttext/f03_items.csv.zip')
items_df = itemlist.items_df

groups_df = pd.read_csv('../data/output/druid_fasttext/clusters.csv.zip', sep=';')

In [52]:
# Adiciona a informação de grupo e ruído no dataframe de itens
items_df = items_df.merge(groups_df, left_on='item_id', right_on='item_id')

# Exclui grupos de ruídos e grupos não particionados
items_df = items_df[(~items_df['grupo'].str.contains('_')) & (~items_df['grupo'].str.contains('-1'))]
items_df['original_desc'] = items_df['original_prep'].apply(lambda x: ' '.join(eval(x)))
items_df.set_index('grupo', inplace=True)

# Quantidade de descrições por grupo
groups_count_df = items_df.groupby('grupo').count().sort_values('item_id', ascending=False)['original_desc']
print(f'Quantidade de grupos considerados: {len(groups_count_df)}')

Quantidade de grupos considerados: 33121


In [53]:
items_df.head()

,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,licitacao,original_prep,ano,item_id,grupo_ruido,item_ruido,original_desc
grupo,,,,,,,,,,,,,,,,,
xyloproct,"['xyloproct', 'pom']",['gr'],['25'],[],[],[],[],28.0600,tubo,XILOPROCT POM. 25 GR,297301,"['xyloproct', 'pom', '25', 'gr']",2014,54,1,1,xyloproct pom 25 gr
promestrieno,"['promestrieno', 'creme', 'vaginal']",[],[],[],[],[],[],66.3267,unidade,PROMESTRIENO CREME VAGINAL,301754,"['promestrieno', 'creme', 'vaginal']",2014,92,1,1,promestrieno creme vaginal
naloxone,['naloxone'],"['mg', 'ml']","['0', '4']",[],[],[],[],13.2500,unidade 1 ampola,"NALOXONE, 0,4MG/ML",32000,"['naloxone', '0', '4', 'mg', 'ml']",2014,101,1,1,naloxone 0 4 mg ml
ete,"['ete', 'para', 'pessoa', 'jose', 'limoeiro']",[],['240'],[],[],[],[],79194.2500,unidade,ETE PARA ATE 240 PESSOAS - SAO JOSE DO LIMOEIRO,2285,"['ete', 'para', '240', 'pessoa', 'jose', 'limo...",2014,142,1,1,ete para 240 pessoa jose limoeiro
zincar,"['zincar', 'sinapi']",[],"['2', '4']",[],[],[],['caixa'],5.4300,unidade,CAIXA ZINCADA 2X4 - SINAPI 83387,33269,"['caixa', 'zincar', '2', 'x', '4', 'sinapi']",2014,146,1,1,caixa zincar 2 x 4 sinapi


In [54]:
def extract_descriptions(group, groups_dict, desc_column):
    if type(desc_column) == str:
        descriptions = {desc_column: 1}
    else:
        descriptions = dict(desc_column.value_counts())
    groups_dict[group]['descriptions'] = descriptions # descrições do grupo
    d_size = [len(d.split(' ')) for d in descriptions.keys()]  # quantidade de tokens por descrição
    groups_dict[group]['sum'] = sum(d_size) # total de tokens do grupo
    groups_dict[group]['median'] = np.median(d_size) # mediana da quantidade de tokens do grupo
    groups_dict[group]['mean'] = np.mean(d_size) # média da quantidade de tokens do grupo
    groups_dict[group]['std'] = np.std(d_size) # desvio padrão da quantidade de tokens do grupo

def extract_tokens(group, groups_dict, desc_column):
    groups_dict[group]['tokens'] = defaultdict(int)
    for desc in desc_column:
        tokens = desc.split(' ')
        for token in tokens:
            groups_dict[group]['tokens'][token] += 1
    groups_dict[group]['tokens'] = {k: v for k, v in sorted(groups_dict[group]['tokens'].items(), key=lambda item: item[1], reverse=True)}

In [55]:
groups_dict = defaultdict(lambda: defaultdict(int))

for group, num_items in groups_count_df.iloc[:1000].iteritems():
    desc_column = items_df.loc[group]['original_desc']
    extract_descriptions(group, groups_dict, desc_column)
    extract_tokens(group, groups_dict, desc_column)

In [98]:
def description_description_strategy(info):
    return list(info['descriptions'].keys())[0]

def get_tokens_frequency(descriptions):
    tokens = set(' '.join(descriptions.keys()).split(' '))
    print(f'** {tokens} **')
    tokens_freq = defaultdict(int)
    for token in tokens:
        for desc in descriptions:
            if token in desc.split(' '):
                tokens_freq[token] += 1
        tokens_freq[token] = round(tokens_freq[token]/len(descriptions), 2)
    tokens_freq = {k:v for k,v in sorted(tokens_freq.items(), key=lambda item: item[1], reverse=True)}
    return list(tokens_freq.items())[:10], np.mean(list(tokens_freq.values()))

def description_token_strategy(info, min_tokens, max_tokens, min_freq):
    tokens = list(info['tokens'].items())
    description = ''
    tokens_freq, mean = get_tokens_frequency(info['descriptions'])
    print(' - 10 tokens mais frequentes nas descrições:', tokens_freq)
    print(' - Média das frequências dos tokens:', mean)
    for i in range(len(tokens_freq[:max_tokens])):
        token = tokens_freq[i][0]
        freq = tokens_freq[i][1]
        if freq >= mean:
            description += f'{token} '
    
#     for i in range(len(tokens[:max_tokens])):
# #         print(tokens[i])
#         key = tokens[i][0]
#         freq = tokens[i][1]/info['sum']
# #         print('freq: ', freq)
#         if freq >= min_freq:
#             description += f'{key} '
    return description

def build_descriptions(groups_dict, min_tokens=3, max_tokens=6, min_freq=0.4):
    for group, info in groups_dict.items():
        tokens = info['tokens']
        
        print(f'## {group} ##')
        print(' - Descrições:', info['descriptions'].keys())
        print(f' - 10 tokens mais frequentes: {list(tokens)[:10]}')
#         print(' - median: ', info['median'])
#         print(' - mean: ', info['mean'])
#         print(' - std: ', info['std'])
        
        # considerando descrição mais frequente
        desc_d = description_description_strategy(info)
        desc_t = description_token_strategy(info, min_tokens, max_tokens, min_freq)
        print(f'ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: {desc_d} ----')
        print(f'ESTRATÉGIA TOKENS MAIS FREQUENTES: {desc_t} ----')
        print('-'*100)
#         print(f'**** {desc_t} ****')

        
build_descriptions(groups_dict)

## bibi ##
 - Descrições: dict_keys(['bibi ir para escola', 'bibi compartilhar coisa', 'bibi ir para cama', 'bibi ir para escola 1 volume editor scipione', 'bibi compartilhar coisa 1 volume editor scipione', 'bibi compartiha coisa', 'bibi fala publico', 'bibi chupa mais dedo', 'bibi corta cabelo', 'bibi comer tudo', 'bibi tomar banho', 'bibi ir para escola editor scipione', 'bibi ir para cama 1 volume editor scipione', 'bibi fonfon fonfonvolante com 8 som diferente pilha', 'bibi compartilhar', 'bibi compartilhar coisa editor scipione'])
 - 10 tokens mais frequentes: ['bibi', 'ir', 'para', 'coisa', 'escola', 'compartilhar', 'editor', 'scipione', 'cama', '1']
** {'corta', 'pilha', 'fonfonvolante', 'cama', 'cabelo', 'diferente', 'volume', 'mais', 'compartilhar', 'scipione', 'ir', 'comer', 'escola', 'fonfon', 'coisa', 'chupa', '1', 'com', 'dedo', 'som', 'publico', 'compartiha', 'banho', 'tomar', 'bibi', 'editor', 'tudo', '8', 'fala', 'para'} **
 - 10 tokens mais frequentes nas descrições: 

## tocinho ##
 - Descrições: dict_keys(['tocinho porco fresco', 'tocinho suino fresco limpo', 'tocinho para torresmo', 'tocinho', 'tocinho suino fresco limpanao temperado bom qualidade com aspecto cor cheiro sabor proprio acondicionado saco transparente resistente embalagem filme pvc plastico contender ideentificacao produto marca fabricante prazer validade carimbo oficial acordo postar ministerio agricultura dipa n 304 22 4 96 105 19 5 99 lei municipal vigilancia sanitario', 'tocinho bacon tipo toucinho', 'tocinho pele suino embalagem contender identificacao produto marca fabricante prazer validade marca carimbo oficial acordo com portaria ministerio agricultura dipa n 304 96 145 98 resolucao anvisa 105 99 fornecedor estar obrigado fornecer embalagem 1 kg', 'tocinho porco fresco 161', 'tocinho barriga suino', 'tocinho com carne'])
 - 10 tokens mais frequentes: ['tocinho', 'fresco', 'suino', 'embalagem', 'marca', 'porco', 'com', 'contender', 'produto', 'fabricante']
** {'contender', 'o

ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: sacarose pa ----
ESTRATÉGIA TOKENS MAIS FREQUENTES: sacarose po com milhar oleo soja  ----
----------------------------------------------------------------------------------------------------
## fotoativo ##
 - Descrições: dict_keys(['resina fotoativo cor b2', 'resina fotoativo cor b1', 'resina fotoativo micro hibrido restauracao anterior seringa 4 g cor opacar a3 referencia charisma metric cerar luna sdi pallis', 'resina fotoativo composto nanar hibrido fotopolimerizavel radiopaco para restauracao direto temp embalagem seringa 4 g cor b3 referencia charisma opal heraeus kulzer metric n cerar hb', 'resina fotoativo cor a3', 'resina fotoativo cor a3 5', 'resina fotoativo composto nanar hibrido fotopolimerizavel radiopaco para restauracao direto seringa 3 5 g possuir pigmento promover efeito camaleao tempo fotoativacao 10 segundo maior igual 1 100 mw cm2 incremento especificar restauracao posterior anterior apresentar baixo contracao stress auto nivel

 - Média das frequências dos tokens: 0.12052980132450333
ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: cerimonial producao evento ----
ESTRATÉGIA TOKENS MAIS FREQUENTES: cerimonial evento producao realizacao montagem para  ----
----------------------------------------------------------------------------------------------------
## natalidade ##
 - Descrições: dict_keys(['kit natalidade', 'kit natalidade 1 banheira maranote para recem nascer saboneteira 2 pacote fralda pano conjunto mijao manda pagao macacao predador toalha banho 4 boby porta fraudar mamadeira 220 ml chuquinha 50 3 par meio 10 algodao 500 soro fisiologico', 'kit natalidade contender 1 banheira plastico 22 litro 3 camiseta pct fralda descartavel 5 fralda pinta borda luva sapato touca pagao curtir longo par meio sabonete 90 gr toalha com capuz', 'kit natalidade 1 banheira saboneteira cx cotonete com 100 un', 'kit natalidade algodao 500 gr banheira cortador unha cotonete fralda descartar kit escova pente lenco umedecer mamadeira gra

## somalium ##
 - Descrições: dict_keys(['somalium 3 mg c 30 cpr', 'somalium 6 mg', 'somalium', 'somalium 3 mg', 'somalium 3 mg cx com 30 comprimido', 'somalium 3 mg fr', 'somalium 6 mg comprimido cx com 30 comprimido', 'somalium comprimido 3 mg'])
 - 10 tokens mais frequentes: ['somalium', 'mg', '3', '30', '6', 'c', 'cpr', 'comprimido', 'cx', 'com']
** {'cx', '3', 'c', '6', 'somalium', 'mg', 'com', 'fr', '30', 'cpr', 'comprimido'} **
 - 10 tokens mais frequentes nas descrições: [('somalium', 1.0), ('mg', 0.88), ('3', 0.62), ('30', 0.38), ('comprimido', 0.38), ('cx', 0.25), ('6', 0.25), ('com', 0.25), ('c', 0.12), ('fr', 0.12)]
 - Média das frequências dos tokens: 0.3972727272727273
ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: somalium 3 mg c 30 cpr ----
ESTRATÉGIA TOKENS MAIS FREQUENTES: somalium mg 3  ----
----------------------------------------------------------------------------------------------------
## condenavel ##
 - Descrições: dict_keys(['resina condenavel z 100', 'resina condenave

 - Média das frequências dos tokens: 0.12200000000000004
ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: ioio ----
ESTRATÉGIA TOKENS MAIS FREQUENTES: ioio acrilico 5 diametro cm plastico  ----
----------------------------------------------------------------------------------------------------
## cauterio ##
 - Descrições: dict_keys(['cauterio passaro', 'kit cauterio', 'cauterio tipo caneta ponta fino', 'cauterio mecir est 1942 shortcut 100', 'kit cauterio com caneta', 'cauterio criocauterio nitrogenio aco inoxidavel 10 ponteira 1 adaptador para agulha 350 ml', 'cauterio tipo caneta ponta largo', 'cauterio para criar cabo caneta ponta', 'cauterio bipolar hst', 'cauterio alto frequencia caf', 'cauterio', 'cauterio bisturi eletronico', 'cauterio bisturi', 'cauterio 110 220 vac 60 hz 10 consumo maximo 170 w fusivel vidro 5 x 20 tipo rapido 3 para 1 potencia maximo saida 150 frequencia trabalho 4 mhz 121 192 157 mm alp', 'cauterio tipo eletrocauterio pot encia coagulacao intensidade regulavel controle

 - Média das frequências dos tokens: 0.27891304347826085
ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: cera rosa 7 wilson comum 18 x 1 225 g 7554 ----
ESTRATÉGIA TOKENS MAIS FREQUENTES: wilson cera 225 rosa 7 com  ----
----------------------------------------------------------------------------------------------------
## parvovirus ##
 - Descrições: dict_keys(['parvovirus 19 igg', 'parvovirus 19 igm', 'parvovirus igm 19', 'parvovirus 19', 'parvovirus igm', 'parvovirus igg', 'parvovirus 19 igg anticorpo anti'])
 - 10 tokens mais frequentes: ['parvovirus', '19', 'igg', 'igm', 'anticorpo', 'anti']
** {'parvovirus', 'igm', 'anti', 'anticorpo', '19', 'igg'} **
 - 10 tokens mais frequentes nas descrições: [('parvovirus', 1.0), ('19', 0.71), ('igm', 0.43), ('igg', 0.43), ('anti', 0.14), ('anticorpo', 0.14)]
 - Média das frequências dos tokens: 0.4750000000000001
ESTRATÉGIA DESCRIÇÃO MAIS FREQUENTE: parvovirus 19 igg ----
ESTRATÉGIA TOKENS MAIS FREQUENTES: parvovirus 19  ----
--------------------------

In [88]:
for group, descriptions in groups_descriptions.items():
    descriptions = list(descriptions)
    print(f'## Grupo {group} ##')
    print(f'Descrição mais frequente: {descriptions[0]}')
    print(f'Próximas 5 descrições mais frequentes: {descriptions[1:6]}')
    print('-'*100)

NameError: name 'groups_descriptions' is not defined

In [ ]:
# distribuição da quantidade de tokens nas descrições mais frequentes
# distribuição da proporção das descrições mais frequentes
# média da quantidade de tokens nas descrições